# Imports

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import linregress

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, BatchNormalization,
    Bidirectional, LayerNormalization, MultiHeadAttention
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import Huber

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
pd.options.plotting.backend = "plotly"

# Data load

In [ ]:
url = "https://storage.googleapis.com/edulabs-public-datasets/powerconsumption.csv.zip"

In [ ]:
df = pd.read_csv(url, compression="zip")

# Feature engineering

In [ ]:
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%m/%d/%Y %H:%M')


df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Month'] = df['Datetime'].dt.month
df['Date'] = df['Datetime'].dt.date

#Data processing

In [ ]:
features = ['Hour', 'DayOfWeek', 'Month', 'Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows']
labels = ['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']
scaler_X = MinMaxScaler()
df[features] = scaler_X.fit_transform(df[features])


In [ ]:
X = df[features].values
y = df[labels].values
print("X shape:", X.shape, "y shape:", y.shape)

# Prepare LSTM sequences

In [ ]:
timesteps = 24

In [ ]:

X_seq, y_seq = [], []
for i in range(timesteps, len(X)):
    X_seq.append(X[i-timesteps:i])
    y_seq.append(y[i])
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)
print("X_seq shape:", X_seq.shape, "y_seq shape:", y_seq.shape)

# Split: 70% train, 15% val, 15% test

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.30, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)
print("Train shapes:", X_train.shape, y_train.shape)
print("Val shapes:", X_val.shape, y_val.shape)
print("Test shapes:", X_test.shape, y_test.shape)

# Build the model

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(64, return_sequences=True),
    LSTM(32, return_sequences=False),
    LayerNormalization(),
    Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.2),
    Dense(3, activation=None),
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# Train the model

In [ ]:
# prompt: train the model

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

history = model.fit(X_train, y_train,
                    epochs=2,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, reduce_lr])